In [120]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

from qiskit.algorithms.optimizers import ADAM, SPSA
from qiskit.circuit.library import TwoLocal, ZZFeatureMap
from qiskit.primitives import BackendEstimator, BackendSampler
from qiskit.quantum_info import Pauli
from qiskit_aer import AerSimulator
from qiskit_algorithms.gradients import ParamShiftEstimatorGradient, ParamShiftSamplerGradient

from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN, SamplerQNN
from qiskit_machine_learning.utils.loss_functions import CrossEntropyLoss

In [121]:
qasm_sim = AerSimulator()
SEED = 123

In [122]:
#create data set
from numpy import random
datasets = random.rand(100, 3, 2) #100 sets of 3 points with x,y coords 0-1 as float

print(datasets[0])

num_classes = 3 # order of circuit 1,2,3.  or should this be all possible options eg 1,2,3 2,3,1?
num_features = 6 # x,y coords, 3 points


[[0.52724478 0.60827797]
 [0.48558892 0.69451112]
 [0.58833792 0.15787654]]


In [128]:
from itertools import permutations

y = []

#solve clasically to assign classes (order), in then end check vs this order / reversed
for points in datasets: #remove number just doing for 1 for now
    
    perms = permutations(points)
    minL = 100.0 #arbitarily high so first point always smaller
    shortestperm = [] #shortest permuation
    
    for perm in perms:
        pflat = np.array(perm).flatten().tolist() #flatten list of arrays to list
        pflatrev = np.array(perm[::-1]).flatten().tolist() #invert list of arrays then flatten to list
        
        if pflat <= pflatrev: #each entry has a reversed counterpart, compare lexographically and only takes 'lowest'
            pathL = 0 #reset path length
            
            for i in range(len(perm)-1):
                pathL += np.sqrt((perm[i][0]-perm[i+1][0])**2 + (perm[i][1]-perm[i+1][1])**2) #add up path lengths
                
            if pathL < minL: #compare to current min distance, and if smaller set new min + shortest permutation
                minL = pathL
                shortestperm = perm

    shortestperm = np.array(shortestperm)
    
    yi = np.zeros(num_classes)
    for i in range(num_classes):
        yi[np.logical_and(points[:,0] == shortestperm[i][0], points[:,1] == shortestperm[i][1])] = i
    
    y.append(yi)
        

In [129]:
#print(y) 
#print(datasets)

[array([1., 0., 2.]), array([2., 1., 0.]), array([2., 0., 1.]), array([2., 1., 0.]), array([1., 0., 2.]), array([0., 2., 1.]), array([0., 1., 2.]), array([1., 2., 0.]), array([1., 0., 2.]), array([1., 0., 2.]), array([1., 2., 0.]), array([1., 2., 0.]), array([0., 2., 1.]), array([1., 2., 0.]), array([2., 0., 1.]), array([2., 0., 1.]), array([1., 2., 0.]), array([2., 1., 0.]), array([2., 0., 1.]), array([1., 0., 2.]), array([1., 2., 0.]), array([2., 0., 1.]), array([1., 2., 0.]), array([2., 1., 0.]), array([1., 2., 0.]), array([0., 1., 2.]), array([0., 1., 2.]), array([1., 0., 2.]), array([1., 0., 2.]), array([0., 1., 2.]), array([2., 1., 0.]), array([0., 1., 2.]), array([2., 0., 1.]), array([2., 1., 0.]), array([0., 1., 2.]), array([1., 0., 2.]), array([0., 2., 1.]), array([2., 0., 1.]), array([2., 1., 0.]), array([0., 2., 1.]), array([1., 0., 2.]), array([0., 2., 1.]), array([1., 0., 2.]), array([1., 0., 2.]), array([1., 0., 2.]), array([1., 0., 2.]), array([0., 1., 2.]), array([2., 1

In [130]:
test_ratio = 0.2
x_train, x_test, y_train, y_test = train_test_split(datasets, y, test_size=test_ratio, random_state=SEED, stratify=y)

print(f"Train set size: {len(x_train)}")
print(f"Test set size:  {len(x_test)}")

Train set size: 80
Test set size:  20
